Для реализации выбран язык Python.

Используемые библиотеки:

In [1]:
from collections import namedtuple
from abc import ABC, abstractmethod
from pprint import pprint
import pandas as pd

import copy
import math
import re
import numpy as np
import networkx as nx
import random

# Параметры моделирования
Пусть топология графа, по которому двигаются машинки, задана в следующей таблице:

In [2]:
graph = [
    [1, True, False, '2(10), 3(20)'],
    [2, False, False, '4(30)'],
    [3, False, False, '4(20)'],
    [4, False, False, '5(10)'],
    [5, False, True, '']
]
pd.DataFrame(graph, columns=['ID вершины', 'Есть исток?', 'Есть сток?', 'Связи'])

,ID вершины,Есть исток?,Есть сток?,Связи
0,1,True,False,"2(10), 3(20)"
1,2,False,False,4(30)
2,3,False,False,4(20)
3,4,False,False,5(10)
4,5,False,True,


Нотация связи следующая: `<id цели>(<время прохождения>){, <id цели>(<время прохождения>)}`.

Время прохождения задано для машинки. Время прохождения для сигнала будет считаться по формуле =`AX(1, FLOOR(<время прохождения> / 10))`.

Определим следующие свойства для моделируемой сети:
- Каждая вершина с истоком каждый такт сети имеет одинаковую вероятность добавить машинку в сеть (`cap_prob`), пока общее число машинок, добавленных в сеть, не превысит опредленного числа (`total_cars`).
- Каждая машинка, когда имеет такую возможность, имеет вероятность отправить сообщение (`msg_prob`) любой другой машинке, находящейся в сети.
- Для маршрутизации сообщений выбран вариант 1 из соответствующего раздела.
- Каждая машинка при появлении в сети заранее знает, в какую вершину она хочет приехать. Путь машинки опредляется как кратчайший до цели.
- Сеть будет смоделирована числа тактов `num_tacts`.


In [3]:
props = {
    'total_cars': 10,
    'car_prob': 0.3,
    'msg_prob': 1,
    'num_tacts': 1000
}

# Создание графа

Для более удобной работы граф, заданный в соответствующей таблице в предыдущем пункте, сконвертирован в соответствующую структуру пакета NetworkX.

In [4]:
def construct_graph(graph):
    G = nx.Graph()
    for id_, is_source, is_sink, connections in graph:
        G.add_node(id_, is_source=is_source == '+', is_sink=is_sink == '+')

        for c in connections.split(','):
            if len(c) == 0:
                continue
            match = re.search(r'(?P<target>\d+)\((?P<distance>\d+)\)', c)
            G.add_edge(
                id_,
                int(
                    match.group('target'),
                ),
                distance=int(match.group('distance'))
            )
    return G

G = construct_graph(graph)

Вывод графа в формате списка смежности:


In [5]:
pprint(nx.to_dict_of_dicts(G))

{1: {2: {'distance': 10}, 3: {'distance': 20}},
 2: {1: {'distance': 10}, 4: {'distance': 30}},
 3: {1: {'distance': 20}, 4: {'distance': 20}},
 4: {2: {'distance': 30}, 3: {'distance': 20}, 5: {'distance': 10}},
 5: {4: {'distance': 10}}}


# Объекты сети

Суперкласс для объектов содержит единственный параметр - последний такт сети, когда объект был обработан. Таким образом, объект не будет обработан несколько раз в одном такте при перемещении по сети.

In [6]:
class DSNObject:
    def __init__(self):
        self.tact = 0
        self._hash = random.randint(0, 10**12)

    def set_processed(self, tact):
        self.tact = tact

    def is_processed(self, tact):
        return self.tact >= tact

    def __hash__(self):
        return self._hash

Объект машинки:

In [7]:
class Car(DSNObject):
    def __init__(self, id, creation_time, target_id):
        """Конструктор

        :param id: id машинки
        :param creation_time: так создания машинки
        :param target_id: id вершины, в которую движется машинка
        """
        super().__init__()
        self.id = id
        self.target_id = target_id
        self.creation_time = creation_time
        self.messages = set()

    def __repr__(self):
        """Представление машинки в текстовом виде
        """
        messages = ";".join([str(m.id) for m in self.messages])
        return f'<Car id={self.id} messages[id]=[{messages}]>'

Объект сообщения:

In [8]:
class Message(DSNObject):
    def __init__(self, id, sender_id, target_id, contents):
        """Конструктор сообщения

        :param id: id сообщения
        :param sender_id: id отправителя
        :param target_id: id получателя
        :param contents: содержимое сообщения
        """
        super().__init__()
        self.id = id
        self.sender_id = sender_id
        self.target_id = target_id
        self.contents = contents

        self.received_ids = set()

    def __repr__(self):
        return f'<Message id={self.id} target_id={self.target_id} sender_id={self.sender_id} contents={self.contents}>'

# Элементы сети

Класс для одной ДСС. Содержит в себе список элементов, свойства, текущий такт и временные параметры (число созданных машинок).

In [9]:
class DSNNetwork:
    def __init__(self, properties, G):
        self.properties = properties
        self.nodes = []
        self.tact = 0
        self.G = G

        self._total_cars = 0

    def process_tact(self):
        for node in self.nodes:
            node.process_tact()
        self.tact += 1

    def __repr__(self):
        res = f'<Network tact={self.tact} nodes=[\n'
        for node in self.nodes:
            res += repr(node) + '\n'
        return res + ']'

Суперкласс для элемента ДСС. Содержит список объектов, список соседей и указатель на объект сети.

In [10]:
class DSNNode(ABC):
    def __init__(self, network):
        self.objects = []
        self.neighbors = []
        self.network = network
        self._hash = random.randint(0, 10**12)

    def add_object(self, object):
        self.objects.append(object)

    def get_neighbors(self, type_=None):
        if type_ is None:
            type_ = DSNNode
        return [n for n in self.neighbors if isinstance(n, type_)]

    def get_unprocessed_objects(self):
        return [o for o in self.objects if not o.is_processed(self.network.tact)]

    def __hash__(self):
        return self._hash

    def __repr__(self):
        res = '<' + self.__class__.__name__
        res += f' id={hash(self)} '
        res += ' objects=['
        if len(self.objects) > 0:
            res += '\n'
        for obj in self.objects:
            res += f'  {repr(obj)}\n'
        res += ']'
        res += ' neighbors[hash]=['
        res += '; '.join([str(hash(obj)) for obj in self.neighbors])
        res += ']>'
        return res

    @abstractmethod
    def process_tact():
        pass

## Исток
Исток генерирует машинки согласно логике, указанной в п. "Структура ДСС". Исток связан только с одним узлом (активным решателем), поэтому машинка будет сразу же добавлена туда.

In [11]:
class Source(DSNNode):
    def _generate_car(self):
        car_id = self.network._total_cars
        self.network._total_cars += 1

        sink_ids = [
            n for n in self.network.G.nodes
            if self.network.G.nodes[n].get('is_sink', False)
        ]
        target_id = random.choice(sink_ids)
        return Car(
            id=car_id, target_id=target_id, creation_time=self.network.tact
        )

    def process_tact(self):
        if (random.random() < self.network.properties['car_prob']) and (
            self.network._total_cars < self.network.properties['total_cars']
        ):
            car = self._generate_car()

            neighbor = self.get_neighbors()[0]
            neighbor.add_object(car)
            car.set_processed(self.network.tact)

## Сток
Сток устраняет все объекты, попавшие в него.

In [12]:
class Sink(DSNNode):
    def process_tact(self):
        for obj in list(self.get_unprocessed_objects()):
            self.objects.remove(obj)

## Пассивный решатель
Пассивный решатель задерживает эелементы сети на количество тактов, определяемое `wait_func`, после чего перенаправляет их в `target_node`

In [13]:
class PassiveResolver(DSNNode):
    def __init__(self, target_node, wait_func, **kwargs):
        super().__init__(**kwargs)
        self.target_node = target_node
        self.objects_waiting = {}
        self.wait_func = wait_func

    def add_object(self, object):
        self.objects.append(object)
        waiting_time = self.wait_func(object)
        self.objects_waiting[object] = waiting_time

    def process_tact(self):
        for obj in self.get_unprocessed_objects():
            self.objects_waiting[obj] -= 1
            if self.objects_waiting[obj] < 0:
                self.objects.remove(obj)
                del self.objects_waiting[obj]
                self.target_node.add_object(obj)
            obj.set_processed(self.network.tact)

    def __repr__(self):
        r = DSNNode.__repr__(self)
        return r[:-1] + f' target_node[hash]={hash(self.target_node)}>'

## Активный решатель
Реализация активного решателя согласно п. "Структура ДСС".

Комментарии к методам:
* `_get_neighbors_by_ids`  
  Для упрощения работы метод для определения id соседних активных, связанных через пассивные решатели или напрямую.
* `_gen_message`  
  В активном решателе происходит генерация сообщений с определенной веротяностью, когда туда попадает машинка:
* `_process_message`
  Активный решатель обрабатывает 2 класса объектов - сообщения и машинки. Для сообщения необходимо отправить его тем соседям, о которых нет информации, что они получили это сообщение:
* `_process_car`  
  Машинку необходимо или направить по кратчайшему пути до пункта назначения, или передать в сток. Также в активном решателе машинка обновляет сообщения.
* `process_tact`  
  Итоговая логика обработки такта.

In [14]:
class ActiveResolver(DSNNode):
    def __init__(self, graph_id, **kwargs):
        super().__init__(**kwargs)
        self.graph_id = graph_id

        self._messages_waiting = {}
        self._messages_sent = set()
        self._neighbors_ids = None

    def __hash__(self):
        return 1000 + self.graph_id

    def _get_neighbors_by_ids(self):
        res = {}
        for n in self.neighbors:
            if isinstance(n, ActiveResolver):
                res[n.id] = n
            elif isinstance(n, PassiveResolver) and n.target_node.graph_id:
                res[n.target_node.graph_id] = n
        return res

    def _gen_message(self, car):
        targets = [i for i in range(self.network._total_cars) if i != car.id]
        if (len(targets) >
            0) and (random.random() < self.network.properties['msg_prob']):
            target_id = random.choice(targets)
            msg = Message(
                id=random.randint(0, 2**24),
                sender_id=car.id,
                target_id=target_id,
                contents='foo'
            )
            self.add_object(msg)

    def _process_message(self, message):
        if not message in self._messages_sent:
            new_received_ids = message.received_ids.union(
                set(self._neighbors_ids.keys())
            )
            for id_, neighbor in self._neighbors_ids.items():
                if id_ not in message.received_ids:
                    new_msg = copy.deepcopy(message)
                    new_msg.received_ids = new_received_ids
                    neighbor.add_object(new_msg)
                    new_msg.set_processed(self.network.tact)
            self._messages_sent.add(message)
            try:
                self._messages_waiting[message.target_id].add(message)
            except KeyError:
                self._messages_waiting[message.target_id] = set([message])
        self.objects.remove(message)

    def _process_car(self, car):
        if car.target_id == self.graph_id:
            sink = self.get_neighbors(Sink)[0]
            self.objects.remove(car)
            sink.add_object(car)
        else:
            path = nx.shortest_path(
                self.network.G,
                source=self.graph_id,
                target=car.target_id,
                weight='distance'
            )
            next_id = path[1]
            next_neighbor = self._neighbors_ids[next_id]
            next_neighbor.add_object(car)
            self.objects.remove(car)
    
        car.set_processed(self.network.tact)
        try:
            car.messages = car.messages.union(self._messages_waiting[car.id])
        except KeyError:
            pass
        self._gen_message(car)

    def process_tact(self):
        if self._neighbors_ids is None:
            self._neighbors_ids = self._get_neighbors_by_ids()
        for obj in self.get_unprocessed_objects():
            if isinstance(obj, Car):
                self._process_car(obj)
        for obj in self.get_unprocessed_objects():
            if isinstance(obj, Message):
                self._process_message(obj)

# Конструирование сети

ДСС конструируется согласно п. "Структура ДСС"

Сначала создаются активные решатели и связываются с истоками и стоками. Затем активные решатели связываются между собой через пассивные решатели.

In [15]:
network = DSNNetwork(props, G)

resolvers = {}

for n in G.nodes:
    node = G.nodes[n]
    resolver = ActiveResolver(graph_id=n, network=network)
    resolvers[n] = resolver

    if node['is_source']:
        source = Source(network=network)
        source.neighbors.append(resolver)
        resolver.neighbors.append(source)
        network.nodes.append(source)

    if node['is_sink']:
        sink = Sink(network=network)
        sink.neighbors.append(sink)
        resolver.neighbors.append(sink)
        network.nodes.append(sink)
    network.nodes.append(resolver)

for e in G.edges:
    id_1, id_2 = e
    res_1, res_2 = resolvers[id_1], resolvers[id_2]
    distance = G.edges[e]['distance']

    def wait_func(obj):
        if isinstance(obj, Car):
            return distance
        else:
            return min(1, math.floor(distance / 10))

    resolver_1 = PassiveResolver(target_node=res_1, wait_func=wait_func, network=network)
    resolver_2 = PassiveResolver(target_node=res_2, wait_func=wait_func, network=network)

    network.nodes.append(resolver_1)
    network.nodes.append(resolver_2)
    resolver_1.neighbors = [res_1, res_2]
    resolver_2.neighbors = [res_1, res_2]
    res_1.neighbors.append(resolver_1)
    res_1.neighbors.append(resolver_2)
    res_2.neighbors.append(resolver_1)
    res_2.neighbors.append(resolver_2)

Можно убедиться, что результат конструирования соотносится с таблицей, указанной в параметрах так, как указано в п. "Структура ДСС"

In [16]:
network

<Network tact=0 nodes=[
<ActiveResolver id=1001  objects=[] neighbors[hash]=[203331673442; 748723156477; 754536355616; 916970249142]>
<ActiveResolver id=1002  objects=[] neighbors[hash]=[203331673442; 748723156477; 696969157607; 102747409110]>
<ActiveResolver id=1003  objects=[] neighbors[hash]=[754536355616; 916970249142; 926029515428; 21977987386]>
<ActiveResolver id=1004  objects=[] neighbors[hash]=[696969157607; 102747409110; 926029515428; 21977987386; 999397235329; 948046945507]>
<ActiveResolver id=1005  objects=[] neighbors[hash]=[999397235329; 948046945507]>
<PassiveResolver id=203331673442  objects=[] neighbors[hash]=[1001; 1002] target_node[hash]=1001>
<PassiveResolver id=748723156477  objects=[] neighbors[hash]=[1001; 1002] target_node[hash]=1002>
<PassiveResolver id=754536355616  objects=[] neighbors[hash]=[1001; 1003] target_node[hash]=1001>
<PassiveResolver id=916970249142  objects=[] neighbors[hash]=[1001; 1003] target_node[hash]=1003>
<PassiveResolver id=696969157607  ob

# Моделирование

Собственно, моделирование. Для отчета по моделированию в каждый такт моделирования сохраняется текстовое представление сети.

In [17]:
def do_modeling(network):
    logs = []
    while network.tact < network.properties['num_tacts']:
        network.process_tact()
        logs.append(repr(network))
    return logs

In [18]:
res = do_modeling(network)
print('\n-----\n'.join(res))

<Network tact=1 nodes=[
<ActiveResolver id=1001  objects=[] neighbors[hash]=[203331673442; 748723156477; 754536355616; 916970249142]>
<ActiveResolver id=1002  objects=[] neighbors[hash]=[203331673442; 748723156477; 696969157607; 102747409110]>
<ActiveResolver id=1003  objects=[] neighbors[hash]=[754536355616; 916970249142; 926029515428; 21977987386]>
<ActiveResolver id=1004  objects=[] neighbors[hash]=[696969157607; 102747409110; 926029515428; 21977987386; 999397235329; 948046945507]>
<ActiveResolver id=1005  objects=[] neighbors[hash]=[999397235329; 948046945507]>
<PassiveResolver id=203331673442  objects=[] neighbors[hash]=[1001; 1002] target_node[hash]=1001>
<PassiveResolver id=748723156477  objects=[] neighbors[hash]=[1001; 1002] target_node[hash]=1002>
<PassiveResolver id=754536355616  objects=[] neighbors[hash]=[1001; 1003] target_node[hash]=1001>
<PassiveResolver id=916970249142  objects=[] neighbors[hash]=[1001; 1003] target_node[hash]=1003>
<PassiveResolver id=696969157607  ob